In [1]:
import os
import numpy as np
import cv2
import pickle
from tqdm import tqdm
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
import torch.nn.utils as utils

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!ls
!tar -xvzf dataset.tar.gz

dataset.tar.gz	my_model_weights_1.pt  sample_data
train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [4]:
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

In [5]:
train_images = torch.tensor(train_images, dtype=torch.float32)
val_images = torch.tensor(val_images, dtype=torch.float32)

train_images = train_images.permute(0, 3, 1, 2)
val_images = val_images.permute(0, 3, 1, 2)

In [6]:
train_dataset = TensorDataset(train_images,
                              torch.tensor(train_labels.squeeze(), dtype=torch.long))
val_dataset = TensorDataset(val_images,
                            torch.tensor(val_labels.squeeze(), dtype=torch.long))

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.model = nn.Sequential(
            # First block: Conv -> ReLU -> Conv -> ReLU -> MaxPool -> Dropout
            nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=True),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=0, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            # Second block: Conv -> ReLU -> Conv -> ReLU -> MaxPool -> Dropout
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=True),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=0, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            # Flatten layer
            nn.Flatten(),

            # Fully connected block: Dense -> ReLU -> Dropout -> Dense -> Softmax
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 5),
        )

    def forward(self, x):
        return self.model(x)

# Running Iterative Pruning

Given some "sparsity" value, we will set the weights of the model for the values to 0 train for n steps and then update the process iteratively.

In [32]:
model = ConvNet()

model.load_state_dict(torch.load('my_model_weights_1.pt'))
model.to(device)

criterion = torch.nn.CrossEntropyLoss()

In [33]:
sparsity = 0.85
N = 500 # number of steps of training
lr = 0.001
num_iter = 100
lamb = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lamb)

In [34]:
# getting lists of weights in the model
weights = []
with torch.no_grad():  # disable gradient tracking for efficiency
  for name, param in model.named_parameters():
    if "weight" in name:  # only apply to weights, skip biases
      weights.append(name)

In [35]:
def train_n_steps(model, train_loader, optimizer, criterion, device,
                  n_steps = N, s = sparsity):


    # setting sparsity
    with torch.no_grad():  # disable gradient tracking for efficiency
      # weight threshold to prune
      threshold = torch.cat([torch.flatten(torch.absolute(model.get_parameter(i))) for i in weights]).quantile(s)
      #print(threshold)
      for name, param in model.named_parameters():
          if "weight" in name:  # only apply to weights, skip biases
              param[torch.absolute(param) < threshold] = 0

    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    steps = 0



    # Progress bar for the training loop
    train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False, total=n_steps)

    for inputs, labels in train_loader_tqdm:

        steps += 1

        if steps > n_steps:
            break

        # Zero the parameter gradients
        optimizer.zero_grad()  # Zero the parameter gradients
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Update tqdm description with current loss and accuracy
        train_loader_tqdm.set_postfix(loss=running_loss / n_steps, accuracy=100 * correct / total)

    train_accuracy = 100 * correct / total
    train_loss = running_loss / n_steps
    return train_loss, train_accuracy

In [36]:
def validate(model, val_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    # Progress bar for the validation loop
    val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)

    with torch.no_grad():  # Disable gradient calculations for validation
        for inputs, labels in val_loader_tqdm:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Track loss and accuracy
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # Update tqdm description with current validation loss and accuracy
            val_loader_tqdm.set_postfix(loss=val_loss / total, accuracy=100 * correct / total)

    val_accuracy = 100 * correct / total
    val_loss = val_loss / len(val_loader)
    return val_loss, val_accuracy

In [37]:
# Main training loop

train_loss_hist = []
train_acc_hist = []
val_loss_hist = []
val_acc_hist = []

for iter in range(num_iter):
    print(f"Pruning Iteration {iter+1}/{num_iter}")

    # Training
    train_loss, train_accuracy = train_n_steps(model, train_loader, optimizer, criterion, device)

    # Validation
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    # Keep History of Output
    train_loss_hist.append(train_loss)
    train_acc_hist.append(train_accuracy)
    val_loss_hist.append(val_loss)
    val_acc_hist.append(val_accuracy)

    # Print epoch results
    print(f'Iter [{iter+1}/{num_iter}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

Pruning Iteration 1/100


Iter [1/100], Train Loss: 1.0582, Train Acc: 58.65%, Val Loss: 0.9663, Val Acc: 61.66%
Pruning Iteration 2/100


Iter [2/100], Train Loss: 1.0327, Train Acc: 59.76%, Val Loss: 0.9187, Val Acc: 64.40%
Pruning Iteration 3/100


Iter [3/100], Train Loss: 1.0195, Train Acc: 60.32%, Val Loss: 0.9439, Val Acc: 63.68%
Pruning Iteration 4/100


Iter [4/100], Train Loss: 0.9988, Train Acc: 61.13%, Val Loss: 0.9274, Val Acc: 62.65%
Pruning Iteration 5/100


...

Iter [96/100], Train Loss: 0.6438, Train Acc: 76.10%, Val Loss: 0.7302, Val Acc: 73.31%
Pruning Iteration 97/100


Iter [97/100], Train Loss: 0.6373, Train Acc: 76.33%, Val Loss: 0.6616, Val Acc: 75.84%
Pruning Iteration 98/100


Iter [98/100], Train Loss: 0.6390, Train Acc: 76.05%, Val Loss: 0.6775, Val Acc: 74.65%
Pruning Iteration 99/100


Iter [99/100], Train Loss: 0.6335, Train Acc: 76.41%, Val Loss: 0.7021, Val Acc: 72.63%
Pruning Iteration 100/100


Iter [100/100], Train Loss: 0.6392, Train Acc: 75.99%, Val Loss: 0.6706, Val Acc: 74.34%


In [38]:
 # forcing low weight connections to be set to 0
with torch.no_grad():  # disable gradient tracking for efficiency
  # weight threshold to prune
  threshold = torch.cat([torch.flatten(torch.absolute(model.get_parameter(i))) for i in weights]).quantile(sparsity)
  for name, param in model.named_parameters():
      if "weight" in name:  # only apply to weights, skip biases
          param[torch.absolute(param) < threshold] = 0

In [39]:
validate(model, val_loader, criterion, device)

(0.6444189482474629, 75.60396039603961)

In [40]:
def calculate_model_sparsity(model):
    total_elements = 0
    zero_elements = 0

    for name, param in model.named_parameters():
        if "weight" in name:  # Consider only trainable parameters
            num_elements = param.numel()
            num_zeros = torch.sum(param <= 1e-4 * 4).item()

            total_elements += num_elements
            zero_elements += num_zeros

    if total_elements == 0:
        return 0.0  # Handle case of no trainable parameters

    sparsity = zero_elements / total_elements
    return sparsity

In [41]:
calculate_model_sparsity(model)

0.9319615550872643

In [42]:
model_name = f'pruning_l1_iterative_sparisity{sparsity}_N{N}_lr{lr}_iter{iter}_forced_sparsity.pt'
torch.save(model.state_dict(), model_name, _use_new_zipfile_serialization=False)

# Creating Pareto Frontiers

In [121]:
from google.colab import files

In [134]:
def create_pareto_point(sparsity_p, reg=1e-3, N_p=500, lr_p=0.001, num_iter_p=100):
  model = ConvNet()

  model.load_state_dict(torch.load('my_model_weights_1.pt'))
  model.to(device)

  criterion = torch.nn.CrossEntropyLoss()

  optim = torch.optim.Adam(model.parameters(), lr=lr_p, weight_decay=reg)
  print(optim)

  # Main training loop

  for iter in range(num_iter_p):
    if iter % 20 == 0:
      print(f"Pruning Iteration {iter+1}/{num_iter_p}")

    # Training
    train_loss, train_accuracy = train_n_steps(model, train_loader, optim, criterion, device,
                                               n_steps = N_p, s = sparsity_p)

    # Validation
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)


    # Print epoch results
    if iter % 20 == 0:
      print(f'Iter [{iter+1}/{num_iter}], '
            f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
            f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

  acc = validate(model, val_loader, criterion, device)
  acc = acc[1]
  obs_sparsity = calculate_model_sparsity(model)


  #model_name = f'pruning_l1_iterative_sparsity{sparsity}_reg_{reg}_N{N}_lr{lr}_iter{iter}.pt'
  #torch.save(model.state_dict(), model_name, _use_new_zipfile_serialization=False)

  print(f'Param: {sparsity_p}, Sparsity:{obs_sparsity}, Acc: {acc}')

  # forcing low weight connections to be set to 0
  with torch.no_grad():  # disable gradient tracking for efficiency
    # weight threshold to prune
    threshold = torch.cat([torch.flatten(torch.absolute(model.get_parameter(i))) for i in weights]).quantile(sparsity_p)
    for name, param in model.named_parameters():
        if "weight" in name:  # only apply to weights, skip biases
            param[torch.absolute(param) < threshold] = 0

  man_set_acc = validate(model, val_loader, criterion, device)
  man_set_acc = man_set_acc[1]
  man_set_obs_sparsity = calculate_model_sparsity(model)



  return sparsity_p, obs_sparsity, acc, man_set_obs_sparsity, man_set_acc

In [135]:
sparsity_levels = [0.99, 0.98, 0.97, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]


In [136]:
df = pd.DataFrame(columns=['sparsity_param', 'sparsity', 'accuracy', 'man_sparsity', 'man_acc'])
l2 = 1e-7
for i in sparsity_levels:
  df.loc[len(df)] = create_pareto_point(sparsity_p = i, reg=l2)
df.to_csv('reg_1e-7_pareto_frontiers.csv', index=False)
files.download('reg_1e-7_pareto_frontiers.csv')

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 1e-07
)
Pruning Iteration 1/100


Iter [1/100], Train Loss: 1.3851, Train Acc: 39.91%, Val Loss: 1.2770, Val Acc: 46.46%


Pruning Iteration 21/100


Iter [21/100], Train Loss: 1.1253, Train Acc: 54.59%, Val Loss: 1.0243, Val Acc: 60.16%


Pruning Iteration 41/100


...

Pruning Iteration 81/100


Iter [81/100], Train Loss: 0.3350, Train Acc: 88.00%, Val Loss: 0.8932, Val Acc: 72.59%


Param: 0.1, Sparsity:0.578260928297401, Acc: 72.5940594059406


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [137]:
df

,sparsity_param,sparsity,accuracy,man_sparsity,man_acc
0,0.99,0.808346,56.990099,0.995362,45.623762
1,0.98,0.716379,67.049505,0.991422,59.247525
2,0.97,0.683686,68.079208,0.987636,63.089109
3,0.95,0.622471,68.871287,0.979744,67.841584
4,0.90,0.569884,70.495050,0.958571,72.594059
5,0.85,0.577297,71.485149,0.937345,72.594059
6,0.80,0.571382,72.792079,0.916223,73.306931
7,0.75,0.565360,73.425743,0.894192,73.386139
8,0.70,0.575779,72.356436,0.871827,73.584158
9,0.65,0.578774,71.207921,0.852031,72.237624


In [138]:
files.download('reg_1e-7_pareto_frontiers.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>